# 0. 変更履歴

* 2019/05/07 初版
* 2019/05/10 第2版
    * pandas print →　display
    * 
    
**5/19追加処理**
* 外れ値を外す。
    * カテゴリ系のデータについては、項目の種類が多いmain_category,countryは、ある程度纏める。
    * 数値系のデータについては、四分位範囲を用いて、外れ値を外す。
* categoryは、Main_categoryと種類が違いそうなので、残す。
* StatusがLiveのものは結果が出ていないので外す。
* 項目の種類が多いものは、いくつかに纏める。

# 1. 通し課題
kaggleのデータセットを用いてモデルを構築し結果を公開することを本講座の通し課題とします。

* 公開先は、 kaggleのKernelsまたはGithubとします
* Slackの所属チャンネルにipynbファイルを直接貼っても良いこととします
* 課題は、以下の2つから選んで下さい
    * 課題① Kickstarter Projects クラウドファンデイングが 成功するか(state)を予測
    * 課題② Car Fuel Consumption 100kmあたりのガソリン消費量(consume)を予測

# 2. 選択課題
以下の課題①を選択する。
* Kickstarter Projects (https://www.kaggle.com/kemical/kickstarter-projects)


# 3. 予測方針
Scikit-learnを用いて、ロジスティック回帰にて実装

# 4. プログラム

## 4-1. データ分析
### 4-1-1. データ、項目の理解
* データの理解
以下の2つのデータがあるが、ks-projects-201612は、古いデータと理解。よって、ks-projects-201801のデータを利用して、予測モデルを構築する。
  * ks-projects-201612
  * ks-projects-201801

* 項目の確認と理解
  * ID：　連番
  * name：　プロジェクト名
  * category：　詳細なカテゴリー
  * main_category：　大きなカテゴリー
  * currency：　pledgedの通貨
  * deadline：　クラウドファンディングの募集締め切り日
  * goal：　クラウドファンディングの目標金額（各国の通貨）
  * launched：　クラウドファンディングの募集開始日
  * pledged：　クラウドファンディングの集まった金額
  * state：　成否
  * backers：　クラウドファンディングした人の数
  * country：　クラウドファンディング主催者の国
  * usd pledged：　クラウドファンディングの集まった金額（KSによってUSDに変換されたもの）
  * usd_pledged_real：　クラウドファンディングの集まった金額（fixer.io apiによってUSDに変換されたもの）
  * usd_goal_real：　クラウドファンディングの目標金額（fixer.io apiによってUSDに変換されたもの）

### 4-1-2. データ分析

In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import log_loss, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix # 回帰問題における性能評価に関する関数
from sklearn.preprocessing import StandardScaler
from scipy.stats import zscore
from scipy import stats

In [46]:
df_company_work = pd.read_csv("ks-projects-201801.csv")
print('*****サンプルデータ*****')
display(df_company_work.head())
print()
print('*****データ統計*****')
display(df_company_work.describe())
print()
print('*****the number of rows and cloumns*****')
display(df_company_work.shape)
print()
print('*****column*****')
display(df_company_work.columns)
print()
print('*****type of column*****')
display(df_company_work.dtypes)
print()
print('*****項目の種類*****')
print('**Contents of ID** ')
print(df_company_work['ID'].unique())
print(df_company_work['ID'].value_counts())
print()
print('**contents of name** ')
print(df_company_work['name'].unique())
print(df_company_work['name'].value_counts())
print()
print('**contents of category** ')
print(df_company_work['category'].unique())
print(df_company_work['category'].value_counts())
print()
print('**contents of main_category** ')
print(df_company_work['main_category'].unique())
print(df_company_work['main_category'].value_counts())
print()
print('**contents of currency** ')
print(df_company_work['currency'].unique())
print(df_company_work['currency'].value_counts())
print()
print('**contents of deadline** ')
print(df_company_work['deadline'].unique())
print(df_company_work['deadline'].value_counts())
print()
print('**contents of goal** ')
print(df_company_work['goal'].unique())
print(df_company_work['goal'].value_counts())
print()
print('**contents of launched** ')
print(df_company_work['launched'].unique())
print(df_company_work['launched'].value_counts())
print()
print('**contents of pledged** ')
print(df_company_work['pledged'].unique())
print(df_company_work['pledged'].value_counts())
print()
print('**contents of state** ')
print(df_company_work['state'].unique())
print(df_company_work['state'].value_counts())
print()
print('**contents of backers** ')
print(df_company_work['backers'].unique())
print(df_company_work['backers'].value_counts())
print()
print('**contents of country** ')
print(df_company_work['country'].unique())
print(df_company_work['country'].value_counts())
print()
print('**contents of usd pledged** ')
print(df_company_work['usd pledged'].unique())
print(df_company_work['usd pledged'].value_counts())
print()
print('**contents of usd_pledged_real** ')
print(df_company_work['usd_pledged_real'].unique())
print(df_company_work['usd_pledged_real'].value_counts())
print()
print('**contents of usd_goal_real** ')
print(df_company_work['usd_goal_real'].unique())
print(df_company_work['usd_goal_real'].value_counts())
print()

print('*****NULLチェック*****')
print('check null',df_company_work.isnull().sum())

print('*****その他*****')
print(df_company_work.groupby(['country','state']).size())


*****サンプルデータ*****


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00



*****データ統計*****


,ID,goal,pledged,backers,usd pledged,usd_pledged_real,usd_goal_real
count,3.786610e+05,3.786610e+05,3.786610e+05,378661.000000,3.748640e+05,3.786610e+05,3.786610e+05
mean,1.074731e+09,4.908079e+04,9.682979e+03,105.617476,7.036729e+03,9.058924e+03,4.545440e+04
std,6.190862e+08,1.183391e+06,9.563601e+04,907.185035,7.863975e+04,9.097334e+04,1.152950e+06
min,5.971000e+03,1.000000e-02,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,1.000000e-02
25%,5.382635e+08,2.000000e+03,3.000000e+01,2.000000,1.698000e+01,3.100000e+01,2.000000e+03
50%,1.075276e+09,5.200000e+03,6.200000e+02,12.000000,3.947200e+02,6.243300e+02,5.500000e+03
75%,1.610149e+09,1.600000e+04,4.076000e+03,56.000000,3.034090e+03,4.050000e+03,1.550000e+04
max,2.147476e+09,1.000000e+08,2.033899e+07,219382.000000,2.033899e+07,2.033899e+07,1.663614e+08



*****the number of rows and cloumns*****


...


*****type of column*****


ID                    int64
name                 object
category             object
main_category        object
currency             object
deadline             object
goal                float64
launched             object
pledged             float64
state                object
backers               int64
country              object
usd pledged         float64
usd_pledged_real    float64
usd_goal_real       float64
dtype: object


*****項目の種類*****
**Contents of ID** 
[1000002330 1000003930 1000004038 ...  999986353  999987933  999988282]
1294469119    1
925977744     1
959180384     1
1003356565    1
926393412     1
809374103     1
1116627353    1
1710038766    1
1138639261    1
1360941471    1
562018722     1
425689509     1
541927920     1
577401272     1
1813483943    1
2106587562    1
57143723      1
1942471085    1
1319623086    1
1088934319    1
217524656     1
1989091761    1
675160465     1
1233047951    1
1453775246    1
450806157     1
839113112     1
62458228      1
2027487605    1
789125494     1
             ..
941767661     1
77713075      1
317903856     1
1457189876    1
184738806     1
1820519415    1
33260506      1
25828679      1
1321948096    1
488525947     1
572169672     1
975391923     1
2125206928    1
65938870      1
1704688581    1
1051943878    1
1520135111    1
1406866376    1
947136390     1
1520108490    1
694887374     1
1637810574    1
364630992     1
1032575953    1
1119601618 

USD    295365
GBP     34132
EUR     17405
CAD     14962
AUD      7950
SEK      1788
MXN      1752
NZD      1475
DKK      1129
CHF       768
NOK       722
HKD       618
SGD       555
JPY        40
Name: currency, dtype: int64

**contents of deadline** 
['2015-10-09' '2017-11-01' '2013-02-26' ... '2009-07-26' '2009-07-30'
 '2009-12-28']
2014-08-08    705
2014-08-10    558
2014-08-07    541
2015-05-01    489
2014-08-09    477
2015-07-01    449
2015-04-01    430
2014-08-15    423
2014-08-31    420
2014-08-14    413
2014-08-20    394
2014-08-13    394
2015-04-30    391
2014-11-01    379
2015-01-01    378
2015-10-01    375
2014-08-16    361
2015-05-31    359
2017-12-01    356
2015-03-01    355
2015-08-01    349
2014-09-12    344
2014-09-07    344
2014-10-31    343
2015-06-01    343
2015-04-03    338
2014-08-11    337
2014-08-17    336
2015-07-31    334
2016-04-01    334
             ... 
2009-08-03      1
2009-08-18      1
2009-09-28      1
2009-08-28      1
2009-07-03      1
2009-05-03     

check null ID                     0
name                   4
category               0
main_category          0
currency               0
deadline               0
goal                   0
launched               0
pledged                0
state                  0
backers                0
country                0
usd pledged         3797
usd_pledged_real       0
usd_goal_real          0
dtype: int64
*****その他*****
country  state     
AT       canceled          91
         failed           378
         live              15
         successful       107
         suspended          6
AU       canceled        1065
         failed          4606
         live              70
         successful      2010
         suspended         88
BE       canceled          74
         failed           371
         live              12
         successful       152
         suspended          8
CA       canceled        2138
         failed          8236
         live             132
         successful      41

## 4-2. 予測モデル 
### 4-2-1 予測モデルの考え方
* 目的変数
  * 目的変数として、結果系の項目である以下の候補が考えられる。今回は、まず成否の判定を行うこととするため、Stateとする。
    * state
    * backers
    * pledged
    * usd_pledged
    * usd_pledged_real
* 説明変数
  * 説明変数については、以下の理由により、項目の削除、統合を行う。
    * 成否の予測に必要のない項目の削除
       * ID
       * Name
    * 成否の予測時に知りえない項目の削除。
       * pledged：　募集金額は、予測時(事前)には知りえない情報のため、説明変数から削除
       * usd pledged：　募集金額は、予測時(事前)には知りえない情報のため、説明変数から削除
       * usd_pledged_real：　募集金額は、予測時(事前)には知りえない情報のため、説明変数から削除
       * backers：　クラウドファンディングした人の数は、予測時(事前)には知りえない情報のため、説明変数から削除
    * 他の項目と意味が同じ,似ている項目を削除
       * goal：　usd_goal_realと単位が異なるだけの項目のため、削除。
       * currency：　countryとほぼ同じ意味合い。また、USDに換算するためだけの項目のため、削除。
    * 統合できる項目
       * launchとdeadlineは、期間として、数値化して表現し、別項目として、管理するため、削除。また、数値化されるため、モデルにも使いやすい。
    * 数値項目でないものを、one-hotベクトル化。
       * main_category　(数が多いので、トップ10のみに削減)
       * main_category　(数が多いので、トップ10のみに削減)
       * state
       * country　(数が多いので、トップ10のみに削減)
    * 0～1に収まっていない項目を標準化
       * usd_goal_real
       * day_diff
   
### 4-2-2 予測モデルのまとめ
* 目的変数
  * stateをone-hotベクトル化したもの
* 説明変数
  * usd_goal_real
  * main_categoryをone-hotベクトル化したもの
  * categoryをone-hotベクトル化したもの
  * countryをone-hotベクトル化したもの
  * deadline - launch
* 条件
  * status = live まだ結果がでていないので、削除。
  
## 4-3. データ前処理

In [47]:
#state:liveは、結果のデータではないので、削除
df_company_work = df_company_work[df_company_work['state'] != 'live']
display(df_company_work.shape)

#数が少ないものはOthersに変更
df_company_work.loc[(df_company_work['category'] != 'Product Design') & (df_company_work['category'] != 'Documentary') & (df_company_work['category'] != 'Music') & (df_company_work['category'] != 'Tabletop Games') & (df_company_work['category'] != 'Shorts') & (df_company_work['category'] != 'Video Games') & (df_company_work['category'] != 'Food') & (df_company_work['category'] != 'Film & Video') & (df_company_work['category'] != 'Fiction') & (df_company_work['category'] != 'Fashion'), 'category' ] = 'Others'
#category_target_list = ['Product Design','Documentary']
#df_company_work.loc[(df_company_work['category'] in category_target_list), 'category' ] = 'Others'
df_company_work.loc[(df_company_work['country'] != 'US') & (df_company_work['country'] != 'GB') & (df_company_work['country'] != 'CA') & (df_company_work['country'] != 'AU') & (df_company_work['country'] != 'DE') & (df_company_work['country'] != 'N,0"') & (df_company_work['country'] != 'FR') & (df_company_work['country'] != 'IT') & (df_company_work['country'] != 'NL') & (df_company_work['country'] != 'ES'), 'country' ] = 'Others'

print('**contents of main_category** ')
print(df_company_work['category'].unique())
print(df_company_work['category'].value_counts())
print('**contents of country** ')
print(df_company_work['country'].unique())
print(df_company_work['country'].value_counts())
display(df_company_work.shape)

(375862, 15)

**contents of main_category** 
['Others' 'Music' 'Film & Video' 'Food' 'Product Design' 'Documentary'
 'Tabletop Games' 'Fashion' 'Fiction' 'Video Games' 'Shorts']
Others            244787
Product Design     22077
Documentary        16082
Music              15647
Tabletop Games     14072
Shorts             12311
Video Games        11744
Food               11454
Film & Video       10054
Fiction             9120
Fashion             8514
Name: category, dtype: int64
**contents of country** 
['GB' 'US' 'CA' 'AU' 'Others' 'IT' 'DE' 'ES' 'N,0"' 'FR' 'NL']
US        290887
GB         33393
CA         14624
Others     10551
AU          7769
DE          4096
N,0"        3796
FR          2887
NL          2833
IT          2802
ES          2224
Name: country, dtype: int64


(375862, 15)

**(確認ポイント)**
* Othersへ変更方法に、もっと簡単なやり方はないか？
    * df_company_work.loc[(df_company_work['category'] != 'Product Design') & (df_company_work['category'] != 'Documentary') & (df_company_work['category'] != 'Music') & (df_company_work['category'] != 'Tabletop Games') & (df_company_work['category'] != 'Shorts') & (df_company_work['category'] != 'Video Games') & (df_company_work['category'] != 'Food') & (df_company_work['category'] != 'Film & Video') & (df_company_work['category'] != 'Fiction') & (df_company_work['category'] != 'Fashion'), 'category' ] = 'Others'


In [48]:
#必要のない項目の削除
df_company_work = df_company_work.drop(['ID','name','currency','goal','pledged','backers','usd pledged','usd_pledged_real'], axis=1)

#one-hotベクトル化
df_company_work = pd.get_dummies(df_company_work, columns = ['category','main_category', 'state', 'country'])
df_company_work = df_company_work.drop(['country_Others','category_Others','state_canceled','state_failed','state_suspended','state_undefined'], axis=1)

#launch deadline 数値化、日数に変換
df_company_work.deadline = pd.to_datetime(df_company_work.deadline, format='%Y-%m-%d')
df_company_work.launched = pd.to_datetime(df_company_work.launched, format='%Y-%m-%d')
df_company_work['day_diff'] = (df_company_work.deadline - df_company_work.launched)
df_company_work.day_diff = df_company_work.day_diff / np.timedelta64(1, 'D')

df_company = df_company_work.drop(['deadline', 'launched'], axis=1)

#前処理後データ確認
print('******前処理後データ確認*****')
display(df_company.head())
display(df_company.describe())
print('the number of rows and cloumns：',df_company.shape)
print('column：',df_company.columns)
print('type of column：',df_company.dtypes)


******前処理後データ確認*****


,usd_goal_real,category_Documentary,category_Fashion,category_Fiction,category_Film & Video,category_Food,category_Music,category_Product Design,category_Shorts,category_Tabletop Games,...,country_CA,country_DE,country_ES,country_FR,country_GB,country_IT,"country_N,0""",country_NL,country_US,day_diff
0,1533.95,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,58.491343
1,30000.00,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,59.802813
2,45000.00,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,44.985532
3,5000.00,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,29.858206
4,19500.00,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,55.642326


,usd_goal_real,category_Documentary,category_Fashion,category_Fiction,category_Film & Video,category_Food,category_Music,category_Product Design,category_Shorts,category_Tabletop Games,...,country_CA,country_DE,country_ES,country_FR,country_GB,country_IT,"country_N,0""",country_NL,country_US,day_diff
count,3.758620e+05,375862.000000,375862.000000,375862.000000,375862.000000,375862.000000,375862.000000,375862.000000,375862.000000,375862.000000,...,375862.000000,375862.000000,375862.000000,375862.000000,375862.000000,375862.000000,375862.000000,375862.000000,375862.000000,375862.000000
mean,4.532720e+04,0.042787,0.022652,0.024264,0.026749,0.030474,0.041630,0.058737,0.032754,0.037439,...,0.038908,0.010898,0.005917,0.007681,0.088844,0.007455,0.010099,0.007537,0.773920,33.875787
std,1.145865e+06,0.202377,0.148791,0.153869,0.161350,0.171888,0.199742,0.235132,0.177992,0.189836,...,0.193376,0.103821,0.076695,0.087304,0.284518,0.086019,0.099987,0.086490,0.418292,66.147023
min,1.000000e-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005058
25%,2.000000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,29.096343
50%,5.480000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,29.683582
75%,1.550000e+04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,36.316120
max,1.663614e+08,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,16738.958333


the number of rows and cloumns： (375862, 38)
column： Index(['usd_goal_real', 'category_Documentary', 'category_Fashion',
       'category_Fiction', 'category_Film & Video', 'category_Food',
       'category_Music', 'category_Product Design', 'category_Shorts',
       'category_Tabletop Games', 'category_Video Games', 'main_category_Art',
       'main_category_Comics', 'main_category_Crafts', 'main_category_Dance',
       'main_category_Design', 'main_category_Fashion',
       'main_category_Film & Video', 'main_category_Food',
       'main_category_Games', 'main_category_Journalism',
       'main_category_Music', 'main_category_Photography',
       'main_category_Publishing', 'main_category_Technology',
       'main_category_Theater', 'state_successful', 'country_AU', 'country_CA',
       'country_DE', 'country_ES', 'country_FR', 'country_GB', 'country_IT',
       'country_N,0"', 'country_NL', 'country_US', 'day_diff'],
      dtype='object')
type of column： usd_goal_real               

## 4-4. 訓練データ、テストデータ作成

In [49]:
#訓練・テストデータ作成
df_company_train, df_company_test = train_test_split(df_company, test_size=0.3)
display(df_company_train.describe())
display(df_company_test.describe())

,usd_goal_real,category_Documentary,category_Fashion,category_Fiction,category_Film & Video,category_Food,category_Music,category_Product Design,category_Shorts,category_Tabletop Games,...,country_CA,country_DE,country_ES,country_FR,country_GB,country_IT,"country_N,0""",country_NL,country_US,day_diff
count,2.631030e+05,263103.000000,263103.000000,263103.000000,263103.000000,263103.000000,263103.000000,263103.000000,263103.000000,263103.000000,...,263103.000000,263103.000000,263103.000000,263103.000000,263103.000000,263103.000000,263103.000000,263103.000000,263103.000000,263103.000000
mean,4.595642e+04,0.042755,0.022702,0.024367,0.026598,0.030631,0.041330,0.059057,0.032584,0.037510,...,0.039008,0.010901,0.005899,0.007788,0.088882,0.007248,0.009859,0.007602,0.774366,33.875960
std,1.181689e+06,0.202305,0.148952,0.154186,0.160906,0.172315,0.199052,0.235731,0.177546,0.190008,...,0.193613,0.103836,0.076577,0.087905,0.284573,0.084827,0.098803,0.086855,0.418000,67.167770
min,1.000000e-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005058
25%,2.000000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,29.096400
50%,5.500000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,29.680729
75%,1.564015e+04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,36.248814
max,1.663614e+08,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,16738.958333


,usd_goal_real,category_Documentary,category_Fashion,category_Fiction,category_Film & Video,category_Food,category_Music,category_Product Design,category_Shorts,category_Tabletop Games,...,country_CA,country_DE,country_ES,country_FR,country_GB,country_IT,"country_N,0""",country_NL,country_US,day_diff
count,1.127590e+05,112759.000000,112759.000000,112759.000000,112759.000000,112759.000000,112759.000000,112759.000000,112759.00000,112759.000000,...,112759.000000,112759.000000,112759.000000,112759.000000,112759.000000,112759.000000,112759.000000,112759.000000,112759.000000,112759.000000
mean,4.385904e+04,0.042861,0.022535,0.024025,0.027102,0.030108,0.042329,0.057991,0.03315,0.037274,...,0.038675,0.010890,0.005960,0.007432,0.088756,0.007937,0.010660,0.007387,0.772878,33.875384
std,1.057572e+06,0.202545,0.148416,0.153126,0.162382,0.170887,0.201340,0.233727,0.17903,0.189434,...,0.192821,0.103788,0.076968,0.085887,0.284392,0.088738,0.102695,0.085633,0.418974,63.702010
min,1.000000e-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.035498
25%,2.000000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,29.096209
50%,5.330490e+03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,29.689653
75%,1.503517e+04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,36.662176
max,1.513959e+08,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,14866.958333


## 4-5. 訓練
### 4-5-1. 訓練データの前処理

In [50]:
# 外れ値削除

#usd_goal_real
#四分位範囲を計算
usd_goal_real_q1 = stats.scoreatpercentile(df_company_train.usd_goal_real, 25) #第一四分位数（=25パーセンタイル）
usd_goal_real_q3 = stats.scoreatpercentile(df_company_train.usd_goal_real, 75) #第三四分位数（=75パーセンタイル）
usd_goal_real_iqr = usd_goal_real_q3 - usd_goal_real_q1 #四分位範囲

#外れ値の範囲を計算する
usd_goal_real_iqr_min = usd_goal_real_q1 - (usd_goal_real_iqr) * 1.5 #第一四分位数 から四分位範囲（iqr*1.5）を引き算。
usd_goal_real_iqr_max = usd_goal_real_q3 + (usd_goal_real_iqr) * 1.5 #第一四分位数 から四分位範囲（iqr*1.5）を。

#print(usd_goal_real_iqr_min)
#print(usd_goal_real_iqr_max)

#範囲から外れている値を除く
df_company_train = df_company_train[df_company_train['usd_goal_real'] < usd_goal_real_iqr_max]
df_company_train = df_company_train[df_company_train['usd_goal_real'] > usd_goal_real_iqr_min]
display(df_company_train.describe())

#day_diff
#四分位範囲を計算
day_diff_q1 = stats.scoreatpercentile(df_company_train.day_diff, 25) #第一四分位数（=25パーセンタイル）
day_diff_q3 = stats.scoreatpercentile(df_company_train.day_diff, 75) #第三四分位数（=75パーセンタイル）
day_diff_iqr = usd_goal_real_q3 - usd_goal_real_q1 #四分位範囲

#外れ値の範囲を計算する
day_diff_iqr_min = day_diff_q1 - (day_diff_iqr) * 1.5 #第一四分位数 から四分位範囲（iqr*1.5）を引き算。
day_diff_iqr_max = day_diff_q3 + (day_diff_iqr) * 1.5 #第一四分位数 から四分位範囲（iqr*1.5）を。

#print(usd_goal_real_iqr_min)
#print(usd_goal_real_iqr_max)

#範囲から外れている値を除く
#df_company = df_company.query('usd_goal_real < usd_goal_real_iqr_max')
df_company_train = df_company_train[df_company_train['day_diff'] < day_diff_iqr_max]
df_company_train = df_company_train[df_company_train['day_diff'] > day_diff_iqr_min]
display(df_company_train.describe())




,usd_goal_real,category_Documentary,category_Fashion,category_Fiction,category_Film & Video,category_Food,category_Music,category_Product Design,category_Shorts,category_Tabletop Games,...,country_CA,country_DE,country_ES,country_FR,country_GB,country_IT,"country_N,0""",country_NL,country_US,day_diff
count,231675.000000,231675.000000,231675.000000,231675.000000,231675.000000,231675.000000,231675.000000,231675.000000,231675.000000,231675.000000,...,231675.000000,231675.000000,231675.000000,231675.000000,231675.000000,231675.000000,231675.000000,231675.000000,231675.000000,231675.000000
mean,7703.274613,0.039465,0.023965,0.026757,0.024806,0.030154,0.044623,0.053648,0.036137,0.039020,...,0.038114,0.009906,0.005547,0.007312,0.088460,0.006440,0.011175,0.006863,0.779022,33.419767
std,8161.527417,0.194698,0.152939,0.161374,0.155535,0.171012,0.206475,0.225323,0.186631,0.193643,...,0.191471,0.099035,0.074269,0.085197,0.283964,0.079991,0.105120,0.082559,0.414906,71.417746
min,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005058
25%,1750.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,29.075984
50%,5000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,29.609444
75%,10000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,34.897118
max,36095.870000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,16738.958333


,usd_goal_real,category_Documentary,category_Fashion,category_Fiction,category_Film & Video,category_Food,category_Music,category_Product Design,category_Shorts,category_Tabletop Games,...,country_CA,country_DE,country_ES,country_FR,country_GB,country_IT,"country_N,0""",country_NL,country_US,day_diff
count,231675.000000,231675.000000,231675.000000,231675.000000,231675.000000,231675.000000,231675.000000,231675.000000,231675.000000,231675.000000,...,231675.000000,231675.000000,231675.000000,231675.000000,231675.000000,231675.000000,231675.000000,231675.000000,231675.000000,231675.000000
mean,7703.274613,0.039465,0.023965,0.026757,0.024806,0.030154,0.044623,0.053648,0.036137,0.039020,...,0.038114,0.009906,0.005547,0.007312,0.088460,0.006440,0.011175,0.006863,0.779022,33.419767
std,8161.527417,0.194698,0.152939,0.161374,0.155535,0.171012,0.206475,0.225323,0.186631,0.193643,...,0.191471,0.099035,0.074269,0.085197,0.283964,0.079991,0.105120,0.082559,0.414906,71.417746
min,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005058
25%,1750.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,29.075984
50%,5000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,29.609444
75%,10000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,34.897118
max,36095.870000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,16738.958333


In [52]:
#標準化
df_company_train.usd_goal_real = zscore(df_company_train.usd_goal_real)
df_company_train.day_diff = zscore(df_company_train.day_diff)

#前処理後データ確認
print('******前処理後データ確認*****')
display(df_company_train)
#print(df_company.head())
#print(df_company.describe())
#print('the number of rows and cloumns：',df_company.shape)
#print('column：',df_company.columns)
#print('type of column：',df_company.dtypes)

******前処理後データ確認*****


,usd_goal_real,category_Documentary,category_Fashion,category_Fiction,category_Film & Video,category_Food,category_Music,category_Product Design,category_Shorts,category_Tabletop Games,...,country_CA,country_DE,country_ES,country_FR,country_GB,country_IT,"country_N,0""",country_NL,country_US,day_diff
193807,2.119305,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0.149885
326798,0.281409,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,-0.135331
142004,0.281409,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,-0.056374
253067,-0.153445,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,-0.194112
117566,-0.576275,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,-0.055316
34065,-0.086170,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0.020691
211648,3.049462,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.370463
36629,1.200357,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0.108952
368353,-0.331222,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,-0.058511
351643,-0.576275,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,-0.048408


In [53]:
### 4-5-2. 訓練
#説明変数と目的変数を分ける。
#係数を求める
y = df_company_train["state_successful"].values
X = df_company_train.drop(['state_successful'], axis=1).values

clf = SGDClassifier(loss = 'log', penalty='none', max_iter=10000, fit_intercept=True, random_state=1234)
clf.fit(X, y)


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=10000, n_iter=None,
       n_jobs=1, penalty='none', power_t=0.5, random_state=1234,
       shuffle=True, tol=None, verbose=0, warm_start=False)

In [54]:
y_pred = clf.predict(X)

# 訓練データの対数尤度を表示
print('訓練データの対数尤度 = {:.3f}'.format(- log_loss(y, y_pred)))

# 訓練データの正答率を計算
accuracy =  accuracy_score(y, y_pred)
print('訓練データの正答率（Accuracy） = {:.3f}%'.format(100 * accuracy))

# 訓練データのPrecision, Recall, F1-scoreを計算
precision, recall, f1_score, _ = precision_recall_fscore_support(y, y_pred)

# 訓練データのカテゴリ「2000万以上」に関するPrecision, Recall, F1-scoreを表示
print('訓練データの適合率（Precision） = {:.3f}%'.format(100 * precision[0]))
print('訓練データの再現率（Recall） = {:.3f}%'.format(100 * recall[0]))
print('訓練データのF1値（F1-score） = {:.3f}%'.format(100 * f1_score[0]))

訓練データの対数尤度 = -11.798
訓練データの正答率（Accuracy） = 65.842%
訓練データの適合率（Precision） = 67.950%
訓練データの再現率（Recall） = 83.953%
訓練データのF1値（F1-score） = 75.108%


### 4-6. テスト
### 4-6-1. テスト前処理

In [55]:
#テストデータ標準化
df_company_test.usd_goal_real = zscore(df_company_test.usd_goal_real)
df_company_test.day_diff = zscore(df_company_test.day_diff)

#前処理後データ確認
print('******前処理後データ確認*****')
display(df_company_test)


******前処理後データ確認*****


C:\Users\tbouc\Anaconda3\lib\site-packages\pandas\core\generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,usd_goal_real,category_Documentary,category_Fashion,category_Fiction,category_Film & Video,category_Food,category_Music,category_Product Design,category_Shorts,category_Tabletop Games,...,country_CA,country_DE,country_ES,country_FR,country_GB,country_IT,"country_N,0""",country_NL,country_US,day_diff
103401,0.019838,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,-0.073745
274965,-0.038635,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,-0.063811
128800,-0.023506,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,-0.215708
11882,-0.038446,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,-0.061188
73510,-0.040999,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,-0.075354
221954,-0.036744,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,-0.046528
22216,-0.040644,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,-0.064068
61719,-0.027288,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,-0.072094
154621,-0.024451,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,-0.076250
340109,-0.015941,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,-0.064277


### 4-6-2. テスト前処理

In [57]:
# テストデータのラベルを予測
y_test = df_company_test["state_successful"].values
X_test = df_company_test.drop(['state_successful'], axis=1).values

y_est = clf.predict(X_test)

# テストデータの対数尤度を表示
print('テストデータの対数尤度 = {:.3f}'.format(- log_loss(y_test, y_est)))

# テストデータの正答率を計算
accuracy =  accuracy_score(y_test, y_est)
print('テストデータの正答率（Accuracy） = {:.3f}%'.format(100 * accuracy))

# テストデータのPrecision, Recall, F1-scoreを計算
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_est)

# テストデータのカテゴリ「2000万以上」に関するPrecision, Recall, F1-scoreを表示
print('テストデータの適合率（Precision） = {:.3f}%'.format(100 * precision[0]))
print('テストデータの再現率（Recall） = {:.3f}%'.format(100 * recall[0]))
print('テストデータのF1値（F1-score） = {:.3f}%'.format(100 * f1_score[0]))

テストデータの対数尤度 = -11.411
テストデータの正答率（Accuracy） = 66.962%
テストデータの適合率（Precision） = 69.886%
テストデータの再現率（Recall） = 85.423%
テストデータのF1値（F1-score） = 76.877%


In [58]:
# テストデータの予測値と正解のクロス集計
conf_mat = pd.DataFrame(confusion_matrix(y_test, y_est), 
                        index=['実際 = successful', '実際 = failed'], 
                        columns=['予測 = successful', '予測 = failed'])
conf_mat

,予測 = successful,予測 = failed
実際 = successful,61929,10568
実際 = failed,26685,13577


# 5. 纏め

外れ値の削除、標準化により、Precision以外は、かなりの改善が見られたが、モデルとしての限界を感じる。次は、NNで実装を試みたい。

* ＜DAY1の結果：liveデータあり、外れ値あり、正規化、種類多い項目のそのまま＞
  * テストデータの対数尤度 = -14.330
  * テストデータの正答率（Accuracy） = 58.512%
  * テストデータの適合率（Precision） = 76.263%
  * テストデータの再現率（Recall） = 51.708%
  * テストデータのF1値（F1-score） = 61.630%

* ＜DAY2の結果：liveデータなし、外れ値削除、標準化、種類多い項目のまとめ＞
  * テストデータの対数尤度 = -11.411
  * テストデータの正答率（Accuracy） = 66.962%
  * テストデータの適合率（Precision） = 69.886%
  * テストデータの再現率（Recall） = 85.423%
  * テストデータのF1値（F1-score） = 76.877%

**(課題・確認ポイント)**
* カテゴリーPrecisionが系の項目が入ってきたときのデータ分析がうまくできていない。
* Precisionが下がった時の対応方法がわからない。